<h1 style="color:#00A6D6;">Introduction to Quantum Cryptography - Jupyter Notebooks</h1>
<h2 style="color:#00A6D6;">Chapter 9: Quantum Cryptography using Untrusted Devices </h2>

Welcome to the new Julia sheet! As usual, we will ask you to use Julia to answer a few exercises. Most importantly, however, the purpose of these Julia sheets is for you to play around and build intuition by exploring and calculating things we do NOT ask you :-) We hope that you take advantage of using Julia this way.

In this Julia Jupyter notebook, we focus on exploring device independent quantum cryptography - by exploring the interrelationship between the violation of the CHSH inequality, and the performance of using entanglement for QKD! The crux of such device independent protocols rests indeed on using CHSH to test the entanglement during test rounds, and then essentially perform entanglement based QKD in the untested rounds. Before you embark on this Julia sheet, is it useful to do the one for Chapters 4 and 8, if you have not already done so.

* <a href="#estate"> An entangled state </a>
* <a href="#chsh"> CHSH using the entangled state </a>

* <a href="#e91"> Entanglement based QKD using the entangled state </a>


The include command that follows will include all the functions that you have used on the previous chapters.

In [ ]:
include("source/main.jl");

<a id="estate"></a>
<h2 style="color:#00A6D6;">An entangled state</h2>
In this Jupyter sheet, we will explore how well we can perform CHSH, and how well we can perform entanglement based QKD, using the very same starting state $\rho_{AB}$ for Alice and Bob. We will set this state here, and then proceed in our investigations. We will pre-set a few possible states to explore below that we will come back to in the exercises, and you are encouraged to try your own! The first is the EPR pair $$|\Psi\rangle = \frac{1}{\sqrt{2}} \left(|00\rangle + |11\rangle\right)\ , $$ which we encoutered many times before.

Second, we will consider another Bell state obtained by applying $X$ to Alice's qubit $$|\Psi\rangle_{10} = \frac{1}{\sqrt{2}}\left(|10\rangle + |01\rangle\right) . $$ 

Finally, we will examine what happens in the case of a Werner state, where we mix the EPR pair with a fully mixed state according to a parameter $p$. This state is given by $$
(1-p) \Phi^+ + p \frac{\mathbb I}{4}\ , 
$$ where $$\Phi^+ = |\Psi\rangle\langle\Psi|.$$

In [ ]:
# The EPR Pair
psi = [1 0 0 1]/sqrt(2);
eprPair = psi' * psi;

# EPR Pair with X applied - a noisy EPR pair!
psiX = [0 1 1 0]/sqrt(2);
eprPairX = psiX'*psiX;

# Werner state for a noise parameter p 
function wernerState(p)
    return p*0.25*I(4) + (1-p)*eprPair;
end

<h3 style="color:#00A6D6;"> Exercise 1</h3>
Let's first investigste whether the states are entangled: 
<ul>
<li> Check whether the EPR pair is indeed entangled.</li>
    <li> Do the same for the EPR pair with X applied to it.</li>
    <li> Investigate whether/when the Werner state is entangled for different values of $p$. Is the Werner state entangled for all $p$? </li>
 </ul>
        

In [ ]:
# Your code goes here - recall that for 2 qubit states we have a convenient function to check entanglement!
check_entangled(eprPair)

<a id="chsh"></a>
<h2 style="color:#00A6D6;">CHSH using the entangled state</h2>

Let's now investigate how well Alice and Bob can perform CHSH, given one of the possible states above. In the exercises below, we will use the standard measurements for CHSH as explained in the Jupyter Sheet for Chapter 4. 

In [ ]:
# Computes the winning probability for CHSH given a specific input state rhoAB
function winningProbCHSH(rhoAB)
    # Set the standard CHSH measurements (see sheet Chapter 4)
    θ_0 = 0;
    θ_1 = 45;
    γ_0 = 22.5;
    γ_1 = -22.5;

    return pwinCHSH(rhoAB,θ_0, θ_1, γ_0, γ_1);
end

<h3 style="color:#00A6D6;"> Exercise 2</h3>

What is the probability that Alice and Bob win the CHSH using 
<ul> 
    <li>a shared state $\rho_{AB}$ given by the EPR pair?</li>
    <li>the shared state $\rho_{AB}$ given by the EPR pair with $X$ applied?</li>
   </ul>
What do you observe? Can you think of a strategy for Alice and Bob in which they can use the second state to obtain the same winning probability that they obtain using the EPR pair?

In [ ]:
# Your code goes here.


<h3 style="color:#00A6D6;">Exercise 3</h3>
Investigate how well Alice and Bob can play the CHSH game using the Werner state with error parameter $p$. What do you observe? For what values of $p$ can Alice and Bob do better than the classical value of 3/4?

In [ ]:
# Build on the code below to investigate!

# Compute the CHSH winning probability for different values of p
function winCHSH(p)
    return winningProbCHSH(wernerState(p));
end

# Plot
# p = range(0,1, length=100);
# winP = winCHSH.(p);
# plot(p,winP)
# xlabel("p")
# ylabel("winning Prob")
# PyPlot.title("CHSH winning probability")
# grid("on")


<h2 style="color:#00A6D6;">E91 Entanglement Based QKD</h2>

Let's now take a look at how well we can perform entanglement based QKD using the very same state as for CHSH above. The code below is adapted from Chapter 8 to let you explore their relationship more easily.

In [ ]:
# Let's define the elements of the BB84 Basis as ket vectors
# The first index indicates the basis and the second the basis element
v0 = [1 0]';
v1 = [0 1]';
w0 = ([1 1]')/sqrt(2);
w1 = ([1 -1]')/sqrt(2);

# The BB84 states. Note that due to Julia indexing from 1 onwards, 1 is the standard basis, and 2 the Hadamard one
# Similarly bit x=0 will correspond to index 1, and x=1 to index 2
BB84 = [[v0] [v1]; [w0] [w1]];

# function to compute the QBERs in X and Z for some state rhoAB for n round

function entangledQKD(rhoAB,n)

    # Build the basis measurements on Alice and Bob's side, we will keep them around for potential future use
    # We will also compute the probabilities of outcomes immediately
    measAB = Dict();
    outcomeProb = Dict();
    for thetaA = 1:2
        for thetaB = 1:2
            outcomeProb[(thetaA,thetaB)] = zeros(4);
            pcount = 1;
            for xA = 1:2
                for xB = 1:2
                    a = BB84[thetaA,xA][1:2];
                    b = BB84[thetaB,xB][1:2]; 

                    # Construct the measurement operator on A and B
                    measAB[(thetaA,xA,thetaB,xB)] = kron(a*a', b*b');

                    # compute the outcome probability on rho that we will sample from later, and add it to a probability vector
                    outcomeProb[(thetaA,thetaB)][pcount] = tr(rhoAB *  measAB[(thetaA,xA,thetaB,xB)]);
                    pcount = pcount + 1;
                end
            end
        end
    end


    # Let's now do QKD
    thetaA = zeros(Int8,n);
    thetaB = zeros(Int8,n);
    errorCount = [0;0];
    sameBasis = [0;0];
    diffBasis = 0;
    sameOutcomeAnyways = 0;

    for i = 1:n
  
        # Alice picks a random basis
        thetaA[i] = rand(1:2);
    
        # Bob picks a random basis
        thetaB[i] = rand(1:2);
    
        # Sample from the outcome distribution
        # outcomes mean 1 -> 00, 2 -> 01, 3 -> 10, 4 -> 11
        rng = Multinomial(1,outcomeProb[(thetaA[i],thetaB[i])]);
        outcome = rand(rng);
        if outcome[1] == 1
            aliceOutcome = 0;
            bobOutcome = 0;
        elseif outcome[2] == 1
            aliceOutcome = 0;
            bobOutcome = 1;
        elseif outcome[3] == 1
            aliceOutcome = 1;
            bobOutcome = 0;
        elseif outcome[4] == 1
            aliceOutcome = 1;
            bobOutcome = 1;
        end
        
        # Print basis used - comment/uncomment as desired
        # print("Alice used basis ", thetaA[i]-1, " with bit ", aliceOutcome ," and Bob measured in basis ", thetaB[i]-1, " and got outcome ", bobOutcome, "\n");
    
        # Compute potential errors
        if (thetaA[i] == thetaB[i])
            # Alice and Bob measured in the same basis
            sameBasis[thetaA[i]] = sameBasis[thetaA[i]] + 1;
            if (aliceOutcome != bobOutcome)
                # this is an error
                errorCount[thetaA[i]] = errorCount[thetaA[i]] + 1;
            end
        end

    end

    # Compute the error rates
    qberZ = errorCount[1]/sameBasis[1];
    qberX = errorCount[2]/sameBasis[2];

    return [qberZ, qberX];
end



<h3 style="color:#00A6D6;"> Exercise 4</h3>
Let's first examine how well Alice and Bob can perform entanglement based QKD!
<ul>
<li> For $\rho_{AB}$ being the EPR pair, what do you observe for the QBER in the standard and the Hadamard basis?</li>
    <li> For $\rho_{AB}$ being the EPR pair with the X applied, what do you observe for the QBERs? Can you explain your observations?</li>
 </ul>
        

In [ ]:
# Run entanglement based QKD with a specific state and n=1000 rounds
output  = entangledQKD(eprPair,1000);

# Let's print the quantum bit error rate in the standard basis

print("QBER in the standard basis: ", output[1], "\n");
print("QBER in the Hadamard basis: ", output[2], "\n");

<h3 style="color:#00A6D6;"> Exercise 5</h3>

Let's now investigate what happens for the Werner state. 
<ul>
    <li>What do you observe for the QBER in Z (standard basis) and X (Hadamard basis) as a function of $p$?</li>
    <li>In Chapter 4 you investigated for which qbers QKD was possible (using one way discussion from Alice to Bob). Use this knowledge to investigate for which values of $p$ for the Werner state, Alice and Bob can still make key.</li>
</ul>

In [ ]:
# Build on the code below to investigate!

# Compute the CHSH winning probability for different values of p
function entQKD(p)
    return entangledQKD(wernerState(p),10000);
end

# Plot
# p = range(0,1, length=100);
# output = entQKD.(p);
# Z = [val[1] for val in output]
# X = [val[2] for val in output]

# plot(p, Z, label="QBER Z")
# plot(p, X, label="QBER X")

# xlabel("p")
# ylabel("QBER")
# PyPlot.title("QBER in Z and X")
# grid("on")
# legend()


<h3 style="color:#00A6D6;"> Exercise 6</h3>
We are now ready to compare CHSH and entanglement based QKD! Investigate the relationship between winning CHSH, and the QBER in Z and X for the Werner state with error parameter $p$. What do you observe? 

In [ ]:
# Your code goes here, where you may want to use the Julia code given above.